In [1]:
import pygame
import math
import Box
import numpy as np
from Super_Classes import Vector, Body, Shape
import Platform
from random import randint
import copy

pygame.init()

display_width = 1000
display_height = 600

gameDisplay = pygame.display.set_mode((display_width,display_height))
pygame.display.set_caption('Game!')

black = (0,0,0)
white = (255,255,255)

clock = pygame.time.Clock()


R = 110
N = 250
path = [np.array([500 + R*math.cos(2*math.pi*x/N),480 + R*math.sin(2*math.pi*x/N)]) for x in range(N)]
d1 = Platform.Moving_Platform([60,10],copy.copy(path),color=(50,50,50),line_color = None ,line_width = 2)
d2 = Platform.Moving_Platform([60,10],path[int(N/2):]+path[:int(N/2)],color=(50,50,50),line_color = None ,line_width = 2)

p0 = Platform.Platform([500,480],[10,10])
p1 = Platform.Platform([100,550],[200,50])
p2 = Platform.Platform([900,550],[200,50])


a = Box.Metal(p1.pos - [-100,p1.size[1]+30])
a1 = Box.Metal(a.pos - [0,a.size[1]*2])
a2 = Box.Metal(a.pos - [a.size[0]*2,0])

b2 = Box.Tnt(path[int(N/2)]-[0,a.size[1]+d1.size[1]])
b2.is_on(d2)
b3 = Box.Wood(path[int(N/2)]-[0,a.size[1]+d1.size[1]+a.size[1]*2])
b3.is_on(b2)
b1 = Box.Nitro(path[0]-[0,a.size[1]+d2.size[1]])
b1.is_on(d1)
b4 = Box.Nitro(path[0]-[0,a.size[1]+d2.size[1]+a.size[1]*2])
b4.is_on(b1)


body_list = [p0,p1,p2,a,a1,a2,d1,d2,b1,b2,b3,b4]
#shape_list = [Shape([(100,100),(20,200),(300,100)],(0,0,0),line_color = (100,0,0)),Shape([(500,100),(120,200),(700,100)],(100,0,100),line_color = (100,0,0))]

body_list += []

#body_list += [Box.Wood((150+k*85, 400)) for k in range(10)]
#body_list[2].vel = np.array([1,-1])

ticker = -1

shifts = [np.array([randint(-10,10)/5,randint(-10,10)/5]) for _ in range(5)]
shifts.append(np.array([0,0]))



while True:

    ### THIS IS NEEDED OR IT WILL CRASH
    for event in pygame.event.get():
        if event.type == 0:
            done = True

    gameDisplay.fill(white)
    
    
    # Earthquake maker!
    if ticker == -1 and False:
        ticker = len(shifts)-2  
    elif ticker>=0:
        ticker -=1
    
        
  
    #for shp in shape_list:
    #    shp.draw(gameDisplay,[0,0])
    
    #pygame.draw.polygon(gameDisplay, (0,0,0), [(100,100),(200,200),(300,100)])
    
    
    for bod in body_list:
        if ticker>=0:
            bod.visual_shift(shifts[ticker]-shifts[ticker+1])
        bod.move()
        bod.draw(gameDisplay,[0,0])
   #     
    
    
    
    
    pygame.display.flip()
    clock.tick(60)
    
    
    
 
    
pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

In [2]:
np.subtract([10,10],[3,4])

array([7, 6])

In [6]:
a = np.array([3,5])
b = np.array([5,2])

all(abs(a-a) < abs(a-b))

True

In [6]:
b.shapes

[<Super_Classes.Shape at 0x10ae58c88>, <Super_Classes.Shape at 0x10ae58cf8>]

In [1]:
path = [i for i in range(10)]

In [3]:
path+path[::-1]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]